# PROGETTO: ANNICHILIMENTO DELL'ANTIIDROGENO

# 1) Gestione dati

In [ ]:
# IMPORT DATI
!rm -rf antihydrogen_test.csv
!rm -rf antihydrogen_valid.csv
!rm -rf antihydrogen_train.csv

!wget http://mlphysics.ics.uci.edu/data/antihydrogen/antihydrogen_test.csv.gz
!wget http://mlphysics.ics.uci.edu/data/antihydrogen/antihydrogen_train.csv.gz
!wget http://mlphysics.ics.uci.edu/data/antihydrogen/antihydrogen_valid.csv.gz

!gzip -d antihydrogen_test.csv.gz
!gzip -d antihydrogen_valid.csv.gz
!gzip -d antihydrogen_train.csv.gz

--2022-07-09 13:04:40--  http://mlphysics.ics.uci.edu/data/antihydrogen/antihydrogen_test.csv.gz
Resolving mlphysics.ics.uci.edu (mlphysics.ics.uci.edu)... 128.195.1.86
Connecting to mlphysics.ics.uci.edu (mlphysics.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6608724 (6.3M) [application/x-gzip]
Saving to: ‘antihydrogen_test.csv.gz’

antihydrogen_test.c 100%[===================>]   6.30M  3.65MB/s    in 1.7s    

2022-07-09 13:04:43 (3.65 MB/s) - ‘antihydrogen_test.csv.gz’ saved [6608724/6608724]

--2022-07-09 13:04:43--  http://mlphysics.ics.uci.edu/data/antihydrogen/antihydrogen_train.csv.gz
Resolving mlphysics.ics.uci.edu (mlphysics.ics.uci.edu)... 128.195.1.86
Connecting to mlphysics.ics.uci.edu (mlphysics.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19840043 (19M) [application/x-gzip]
Saving to: ‘antihydrogen_train.csv.gz’

antihydrogen_train. 100%[===================>]  18.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time

from torchsummary import summary

import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

1.11.0+cu113
0.12.0+cu113


In [ ]:
# check if GPU is avaible and which kind
if torch.cuda.is_available():
  print('Numero di GPU disponibili: ',torch.cuda.device_count())
  for i in range(0,torch.cuda.device_count()):
    print(torch.cuda.get_device_name(0))

# if GPU is avaible: device='cuda', otherwise 'cpu'
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}\n")

Numero di GPU disponibili:  1
Tesla T4
Computation device: cuda



In [ ]:
#Data analysis

#Column 0: Target
#Columns 1-447: Inner-Z, 447 features.
#Columns 448-693: Inner-PHI, 246 features.
#Columns 694-1140: Outer-Z, 447 features.
#Columns 1141-1430: Outer-PHI, 290 features.

Neve = 200000
t0 = time.time()
#Read only Neve of the total events, leaving unchanged the ratio train/test/vali : 60/20/20
#In questo modo non si hanno problemi di RAM
tr_df = pd.read_csv('antihydrogen_train.csv', dtype=np.int8, sep=' ', nrows = Neve) 
val_df = pd.read_csv('antihydrogen_valid.csv', dtype=np.int8, sep=' ', nrows = int(Neve * 0.3))
tes_df = pd.read_csv('antihydrogen_test.csv', dtype=np.int8, sep=' ', nrows = int(Neve*0.3))

print("tempo impiegato:", round((time.time()-t0)/60, 1), "min")
#time to read all the events: 2.6 min
#time to read 200k: 0.6 min

tempo impiegato: 0.3 min


In [ ]:
N_train = len(tr_df.index)
N_vali = len(val_df.index)
N_test = len(tes_df.index)

print("Number of datas training:", N_train)
print("# Number of datas validation:", N_vali)
print("# Number of datas test:", N_test)

Number of datas training: 200000
# Number of datas validation: 60000
# Number of datas test: 60000


In [ ]:
# CREATE NEW COLUMNS FOR THE DF
c0_label = 'Target'
c0_idx = 0
c1_label = 'Inner-Z'
c1_idx_min = c0_idx + 1
c1_idx_max = 447
c2_label = 'Inner-Phi'
c2_idx_min = c1_idx_max + 1
c2_idx_max = 693
c3_label = 'Outer-Z'
c3_idx_min = c2_idx_max + 1
c3_idx_max = 1140
c4_label = 'Outer-Phi'
c4_idx_min = c3_idx_max + 1
c4_idx_max = 1430

colonne = []
colonne.append(c0_label)
c1 = [c1_label + str(i+1) for i in range(0, c1_idx_max - c1_idx_min+1)]
colonne += c1
c2 = [c2_label + str(i+1) for i in range(0, c2_idx_max - c2_idx_min+1)]
colonne += c2
c3 = [c3_label + str(i+1) for i in range(0, c3_idx_max - c3_idx_min+1)]
colonne += c3
c4 = [c4_label + str(i+1) for i in range(0, c4_idx_max - c4_idx_min+1)]
colonne += c4

# SWITCH COLUMNS NAME
tr_df.columns = colonne
val_df.columns = colonne
tes_df.columns = colonne

# SHOW DF
display(tr_df.head())


,Target,Inner-Z1,Inner-Z2,Inner-Z3,Inner-Z4,Inner-Z5,Inner-Z6,Inner-Z7,Inner-Z8,Inner-Z9,...,Outer-Phi281,Outer-Phi282,Outer-Phi283,Outer-Phi284,Outer-Phi285,Outer-Phi286,Outer-Phi287,Outer-Phi288,Outer-Phi289,Outer-Phi290
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# select only the events with label 1
tr_signal = tr_df.loc[tr_df['Target']>0.5]
# select only the events with label 0
tr_background = tr_df.loc[tr_df['Target']<0.5]


# select only the events with label 1
tes_signal = tes_df.loc[tes_df['Target']>0.5]
# select only the events with label 0
tes_background = tes_df.loc[tes_df['Target']<0.5]

# select only the events with label 1
val_signal = val_df.loc[val_df['Target']>0.5]
# select only the events with label 0
val_background = val_df.loc[val_df['Target']<0.5]

print(f'Numero eventi di segnale train {len(tr_signal)}.')
print(f'Numero eventi di fondo train {len(tr_background)}.')
print('\n')

print(f'Numero eventi di segnale test {len(tes_signal)}.')
print(f'Numero eventi di fondo test {len(tes_background)}.')
print('\n')

print(f'Numero eventi di segnale vali {len(val_signal)}.')
print(f'Numero eventi di fondo vali {len(val_background)}.')

Numero eventi di segnale train 98589.
Numero eventi di fondo train 101411.


Numero eventi di segnale test 29415.
Numero eventi di fondo test 30585.


Numero eventi di segnale vali 29491.
Numero eventi di fondo vali 30509.


Slight imbalance in favor of signal events. Given the large number of data, we can undersample by removing random rows

In [ ]:
tr_df = tr_df.drop(tr_df[tr_df.Target.eq(0)].sample(np.abs(len(tr_signal)-len(tr_background))).index)
tes_df = tes_df.drop(tes_df[tes_df.Target.eq(0)].sample(np.abs(len(tes_signal)-len(tes_background))).index)
val_df = val_df.drop(val_df[val_df.Target.eq(0)].sample(np.abs(len(val_signal)-len(val_background))).index)

# select only the events with label 1
tr_signal = tr_df.loc[tr_df['Target']>0.5]
# select only the events with label 0
tr_background = tr_df.loc[tr_df['Target']<0.5]


# select only the events with label 1
tes_signal = tes_df.loc[tes_df['Target']>0.5]
# select only the events with label 0
tes_background = tes_df.loc[tes_df['Target']<0.5]

# select only the events with label 1
val_signal = val_df.loc[val_df['Target']>0.5]
# select only the events with label 0
val_background = val_df.loc[val_df['Target']<0.5]

print(f'Numero eventi di segnale train {len(tr_signal)}.')
print(f'Numero eventi di fondo train {len(tr_background)}.')
print('\n')

print(f'Numero eventi di segnale test {len(tes_signal)}.')
print(f'Numero eventi di fondo test {len(tes_background)}.')
print('\n')

print(f'Numero eventi di segnale vali {len(val_signal)}.')
print(f'Numero eventi di fondo vali {len(val_background)}.')

Numero eventi di segnale train 98589.
Numero eventi di fondo train 98589.


Numero eventi di segnale test 29415.
Numero eventi di fondo test 29415.


Numero eventi di segnale vali 29491.
Numero eventi di fondo vali 29491.


Perfectly balanced

In [ ]:
# DIVIDE THE DATAS IN 5 CATEGORIES: TARGET, INNER-Z, INNER-PHI, OUTER-Z, OUTER-PHI
tr_target = np.array(tr_df[c0_label])
tr_innerZ = np.array(tr_df[c1])
tr_innerPhi = np.array(tr_df[c2])
tr_outerZ = np.array(tr_df[c3])
tr_outerPhi = np.array(tr_df[c4])

tes_target = np.array(tes_df[c0_label])
tes_innerZ = np.array(tes_df[c1])
tes_innerPhi = np.array(tes_df[c2])
tes_outerZ = np.array(tes_df[c3])
tes_outerPhi = np.array(tes_df[c4])

val_target = np.array(val_df[c0_label])
val_innerZ = np.array(val_df[c1])
val_innerPhi = np.array(val_df[c2])
val_outerZ = np.array(val_df[c3])
val_outerPhi = np.array(val_df[c4])

print("shape tr_target:", tr_target.shape)
print("shape tr_innerZ:", tr_innerZ.shape)
print("shape tr_innerPhi:", tr_innerPhi.shape)
print("shape tr_outerZ:", tr_outerZ.shape)
print("shape tr_outerPhi:", tr_outerPhi.shape)
print('\n')

print("shape tes_target:", tes_target.shape)
print("shape tes_innerZ:", tes_innerZ.shape)
print("shape tes_innerPhi:", tes_innerPhi.shape)
print("shape tes_outerZ:", tes_outerZ.shape)
print("shape tes_outerPhi:", tes_outerPhi.shape)
print('\n')

print("shape val_target:", val_target.shape)
print("shape val_innerZ:", val_innerZ.shape)
print("shape val_innerPhi:", val_innerPhi.shape)
print("shape val_outerZ:", val_outerZ.shape)
print("shape val_outerPhi:", val_outerPhi.shape)

shape tr_target: (197178,)
shape tr_innerZ: (197178, 447)
shape tr_innerPhi: (197178, 246)
shape tr_outerZ: (197178, 447)
shape tr_outerPhi: (197178, 290)


shape tes_target: (58830,)
shape tes_innerZ: (58830, 447)
shape tes_innerPhi: (58830, 246)
shape tes_outerZ: (58830, 447)
shape tes_outerPhi: (58830, 290)


shape val_target: (58982,)
shape val_innerZ: (58982, 447)
shape val_innerPhi: (58982, 246)
shape val_outerZ: (58982, 447)
shape val_outerPhi: (58982, 290)


In [ ]:
# Features ranking, we want that tr_outerPhi and tr_innerPhi have the same shape

from sklearn.ensemble import ExtraTreesClassifier


forest = ExtraTreesClassifier(n_estimators=100,
                              random_state=0)


forest.fit(tr_outerPhi, tr_target)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
            axis=0)
indices = np.argsort(importances)[::-1]

In [ ]:
features = list(tr_df.columns)
outerPhi_imp = np.array([features[j+1140] for j in indices[:246]])

#Feature ranking solo sul training set per evitare leakage di informazioni
tr_outerPhi = np.array(tr_df[outerPhi_imp])
tes_outerPhi = np.array(tes_df[outerPhi_imp])
val_outerPhi = np.array(val_df[outerPhi_imp])

print(f'tr_outerPhi shape: {tr_outerPhi.shape}')
print(f'tes_outerPhi shape: {tes_outerPhi.shape}')
print(f'val_outerPhi shape: {val_outerPhi.shape}')

tr_outerPhi shape: (197178, 246)
tes_outerPhi shape: (58830, 246)
val_outerPhi shape: (58982, 246)


In [ ]:
from sklearn.preprocessing import StandardScaler

transformer = StandardScaler()
tr_outerPhi = transformer.fit_transform(tr_outerPhi)
tes_outerPhi = transformer.transform(tes_outerPhi)
val_outerPhi = transformer.transform(val_outerPhi)

transformer = StandardScaler()
tr_innerPhi = transformer.fit_transform(tr_innerPhi)
tes_innerPhi = transformer.transform(tes_innerPhi)
val_innerPhi = transformer.transform(val_innerPhi)

transformer = StandardScaler()
tr_outerZ = transformer.fit_transform(tr_outerZ)
tes_outerZ = transformer.transform(tes_outerZ)
val_outerZ = transformer.transform(val_outerZ)

transformer = StandardScaler()
tr_innerZ = transformer.fit_transform(tr_innerZ)
tes_innerZ = transformer.transform(tes_innerZ)
val_innerZ = transformer.transform(val_innerZ)


In [ ]:
# OBIETTIVO: RIDURRE LA DIMENSIONE DEL CAMPIONE DI OUTER PHI TRAMITE INTERPOLAZIONE LINEARE
#from sklearn.decomposition import PCA
#n_comp = tr_innerPhi[0].shape[0]
#pca_phi = PCA(n_components=n_comp, random_state=None)
#tr_outerPhi = pca_phi.fit_transform(tr_outerPhi)
#tes_outerPhi = pca_phi.fit_transform(tes_outerPhi)
#val_outerPhi = pca_phi.fit_transform(val_outerPhi)

#print(f'tr_outerPhi shape: {tr_outerPhi.shape}')
#print(f'tes_outerPhi shape: {tes_outerPhi.shape}')
#print(f'val_outerPhi shape: {val_outerPhi.shape}')

In [ ]:
tr_Z = []
tes_Z = []
val_Z = []

for i in range(tr_innerZ.shape[0]):
  tr_Z.append(np.vstack([tr_innerZ[i], tr_outerZ[i]]))
tr_Z = np.array(tr_Z)

for i in range(tes_innerZ.shape[0]):
  tes_Z.append(np.vstack([tes_innerZ[i], tes_outerZ[i]]))
tes_Z = np.array(tes_Z)

for i in range(val_innerZ.shape[0]):
  val_Z.append(np.vstack([val_innerZ[i], val_outerZ[i]]))
val_Z = np.array(val_Z)



print(tr_Z.shape)
print(tes_Z.shape)
print(val_Z.shape)

(197178, 2, 447)
(58830, 2, 447)
(58982, 2, 447)


In [ ]:
tr_Phi = []
tes_Phi = []
val_Phi = []

for i in range(tr_outerPhi.shape[0]):
  tr_Phi.append(np.vstack([tr_innerPhi[i], tr_outerPhi[i]]))
tr_Phi = np.array(tr_Phi)

for i in range(tes_outerPhi.shape[0]):
  tes_Phi.append(np.vstack([tes_innerPhi[i], tes_outerPhi[i]]))
tes_Phi = np.array(tes_Phi)

for i in range(val_outerPhi.shape[0]):
  val_Phi.append(np.vstack([val_innerPhi[i], val_outerPhi[i]]))
val_Phi = np.array(val_Phi)


print(tr_Phi.shape)
print(tes_Phi.shape)
print(val_Phi.shape)

(197178, 2, 246)
(58830, 2, 246)
(58982, 2, 246)


In [ ]:
from torchvision.transforms import ToTensor


tr_target = np.array(tr_df[c0_label])
tr_Z = tr_Z.astype('float32')
tr_Phi = tr_Phi.astype('float32')
tr_target = tr_target.astype('int')

tes_target = np.array(tes_df[c0_label])
tes_Z = tes_Z.astype('float32')
tes_Phi = tes_Phi.astype('float32')
tes_target = tes_target.astype('int')

val_target = np.array(val_df[c0_label])
val_Z = val_Z.astype('float32')
val_Phi = val_Phi.astype('float32')
val_target = val_target.astype('int')

print(tr_Z.shape)
print(tr_Phi.shape)
print(tr_target.shape)
print('\n')

print(tes_Z.shape)
print(tes_Phi.shape)
print(tes_target.shape)
print('\n')

print(val_Z.shape)
print(val_Phi.shape)
print(val_target.shape)

(197178, 2, 447)
(197178, 2, 246)
(197178,)


(58830, 2, 447)
(58830, 2, 246)
(58830,)


(58982, 2, 447)
(58982, 2, 246)
(58982,)


In [ ]:
tr_Z = torch.tensor(tr_Z)
tr_Phi = torch.tensor(tr_Phi)
tr_target = torch.tensor(tr_target)

tes_Z = torch.tensor(tes_Z)
tes_Phi = torch.tensor(tes_Phi)
tes_target = torch.tensor(tes_target)

val_Z = torch.tensor(val_Z)
val_Phi = torch.tensor(val_Phi)
val_target = torch.tensor(val_target)


#CREATE torch datasets 
from torch.utils.data import TensorDataset

tr_Dataset = TensorDataset(tr_Phi,tr_Z,tr_target)
tes_Dataset = TensorDataset(tes_Phi,tes_Z,tes_target)
val_Dataset = TensorDataset(val_Phi,val_Z,val_target)

#CREATE DataLoaders
from torch.utils.data import DataLoader
torch.manual_seed(101)

batch_size =  100

tr_dataloader = DataLoader(
    tr_Dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

tes_dataloader = DataLoader(
    tes_Dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

val_dataloader = DataLoader(
    val_Dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)


u, x, y = next(iter(tr_dataloader))
print("Z features shape:", x.shape)
print("Target shape:", y.shape)
print("Phi features shape:", u.shape)

#print(x)
#print(u)

Z features shape: torch.Size([100, 2, 447])
Target shape: torch.Size([100])
Phi features shape: torch.Size([100, 2, 246])


In [ ]:
from numpy.ma.core import outer
from torch import nn
import torch.nn.functional as F
import torch

class myDNN(nn.Module):
  def __init__(self):
    super(myDNN, self).__init__()

    # layers definition

    # first convolutional block
    self.path1_conv1 = nn.Conv1d(in_channels=2, out_channels=8, kernel_size=7)
    self.path1_pool1 = nn.MaxPool1d(kernel_size = 2, stride=2)

    # second convolutional block
    self.path1_conv2 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3)
    self.path1_pool2 = nn.MaxPool1d(kernel_size = 2, stride=2)

    # third convolutional block
    self.path1_conv3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
    self.path1_pool3 = nn.MaxPool1d(kernel_size = 2, stride=2)

    # fourth convolutional block
    self.path1_conv4 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
    self.path1_pool4 = nn.MaxPool1d(kernel_size = 2, stride=2)

    # fifth convolutional block
    self.path1_conv5 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
    self.path1_pool5 = nn.MaxPool1d(kernel_size = 2, stride=2)

    self.path2_conv1 = nn.Conv1d(in_channels=2, out_channels=8, kernel_size=7)
    self.path2_pool1 = nn.MaxPool1d(kernel_size = 2, stride=2)

    # second convolutional block
    self.path2_conv2 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3)
    self.path2_pool2 = nn.MaxPool1d(kernel_size = 2, stride=2)

    # third convolutional block
    self.path2_conv3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
    self.path2_pool3 = nn.MaxPool1d(kernel_size = 2, stride=2)

    # fourth convolutional block
    self.path2_conv4 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
    self.path2_pool4 = nn.MaxPool1d(kernel_size = 2, stride=2)

    # fifth convolutional block
    self.path2_conv5 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
    self.path2_pool5 = nn.MaxPool1d(kernel_size = 2, stride=2)


    self.flatten = nn.Flatten()
    self.drop1 = nn.Dropout(p=0.5)
    self.fc1 = nn.Linear(in_features=2048, out_features=50) 
    self.drop2 = nn.Dropout(p=0.5) #dropout
    self.fc2 = nn.Linear(in_features=50, out_features=25)
    self.fc3 = nn.Linear(in_features=25, out_features=2)


  def forward(self, x, y):
    x = F.relu(self.path1_conv1(x))
    x = self.path1_pool1(x)
    x = F.relu(self.path1_conv2(x))
    x = self.path1_pool2(x)
    x = F.relu(self.path1_conv3(x))
    x = self.path1_pool3(x)
    x = F.relu(self.path1_conv4(x))
    x = self.path1_pool3(x)
    x = F.relu(self.path1_conv5(x))
    x = self.path1_pool5(x)

    y = F.relu(self.path2_conv1(y))
    y = self.path2_pool1(y)
    y = F.relu(self.path2_conv2(y))
    y = self.path2_pool2(y)
    y = F.relu(self.path2_conv3(y))
    y = self.path2_pool3(y)
    y = F.relu(self.path2_conv4(y))
    y = self.path2_pool3(y)
    y = F.relu(self.path2_conv5(y))
    y = self.path2_pool5(y)

    #flatten
    x = self.flatten(x)
    y = self.flatten(y)

    w = torch.cat([x,y],dim=1)
    w = self.drop1(w) #dropout layer
    w = F.relu(self.fc1(w)) #layer fully connected with re lu
    w = self.drop2(w)
    w = F.relu(self.fc2(w)) #layer fully connected with re lu

    w = self.fc3(w) #layer fully connected
    #out = F.log_softmax(w, dim=1) non serve, crossentropy lo fa già lui

    return w

In [ ]:
model = myDNN()
print(model)
from torchsummary import summary
if torch.cuda.is_available():
  summary(model.cuda(), input_size = [(2,246),(2,447)])
else:
  summary(model, input_size = [(2,246),(2,447)])

myDNN(
  (path1_conv1): Conv1d(2, 8, kernel_size=(7,), stride=(1,))
  (path1_pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (path1_conv2): Conv1d(8, 16, kernel_size=(3,), stride=(1,))
  (path1_pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (path1_conv3): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
  (path1_pool3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (path1_conv4): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (path1_pool4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (path1_conv5): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
  (path1_pool5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (path2_conv1): Conv1d(2, 8, kernel_size=(7,), stride=(1,))
  (path2_pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (path2_conv2): Conv1d(8, 16, kernel_size=(3,), stride=(1,))
  (pa

In [ ]:
# testing a batch on the model to see if things work

feat_phi, feat_z, label = next(iter(tr_dataloader))
feat_z=feat_z.to(device) 
feat_phi=feat_phi.to(device)
label=label.to(device)

out = model(feat_z,feat_phi) 

print(out.shape) 
print(out[0]) 

torch.Size([100, 2])
tensor([-0.0400, -0.2173], device='cuda:0', grad_fn=<SelectBackward0>)


In [ ]:
# loss function 
loss_func = nn.CrossEntropyLoss().cuda() #cross entropy loss
# 2 classes -> binary cross entropy

# metric accuracy (not built in in pytorch)
def accuracy(yhat, y):
    preds = torch.max(yhat,1)[1] #predictions == neurons with maximum probability 
    batch_acc = (preds == y).sum()
    return batch_acc

metric_func = accuracy

from torch import optim

opt = optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.999), eps = 1e-8, weight_decay=0) 
scheduler = optim.lr_scheduler.StepLR(opt, 1, gamma=0.99, last_epoch=-1, verbose=False) #ogni 1 epoche il LR è moltiplicato per gamma

In [ ]:
# controlliamo se la GPU è disponibile e nel caso quale tipo di GPU
if torch.cuda.is_available(): # True se ho la GPU
  print('Numero di GPU disponibili: ',torch.cuda.device_count())
  !nvidia-smi
  for i in range(0,torch.cuda.device_count()):
    print(torch.cuda.get_device_name(0))
  

# se la GPU è disponibile setto device='cuda', altrimenti 'cpu
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}\n")

In [ ]:
# model on GPU
model.to(device)
print(next(model.parameters()).device)

cuda:0


In [ ]:
# in pytorch è necessario scriversi il loop di training, cioè il loop sulle epoch di training in cui in ogni 
# epoca si leggono tutti gli eventi del dataset aggiornando i pesi dopo ogni mini-batch


# numero di epoche
epochs = 100

# liste su cui salvare il valore della loss e della metrica ad ogni epoca per poterli graficare in funzione 
# dell'epoca a fine addestramento
hist_loss = []
hist_metric = []
hist_vloss = []
hist_vmetric = []



# loop sulle epoche
for epoch in range(epochs):
    t0 = time.time()
    
    # training step (in cui aggiorniamo i pesi della rete neurale)
    model.train()
    train_loss = 0
    train_metric = 0
    counter = 0

    for inputs1, inputs2, targets in tr_dataloader:  
        counter += 1
        inputs1=inputs1.to(device) 
        inputs2=inputs2.to(device) 
        targets=targets.to(device)

        pred = model(inputs1,inputs2)

        # calcolo loss e metrica
        loss = loss_func(pred, targets)
        with torch.no_grad():
          metric = metric_func(pred, targets)

        # aggiorno la loss e metrica totale
        train_loss += loss.item()
        train_metric += metric.item()

        # backpropagation
        opt.zero_grad() #resetta i gradienti prima di eseguire la backpropagation
        loss.backward() #calcola i gradienti della loss
        opt.step() #aggiorna i pesi

    train_loss /= counter
    train_metric /= (counter*batch_size)
    hist_loss.append(train_loss)
    hist_metric.append(train_metric)


    
        # validation step (non vengono aggiornati i pesi), si misurano le performance sul validation
    model.eval()
    vali_loss = 0
    vali_metric = 0
    counter = 0
    with torch.no_grad(): #evita che si aggiornino i pesi
      for inputs1, inputs2, targets in val_dataloader:  
        counter +=1
        inputs1=inputs1.to(device) 
        inputs2=inputs2.to(device) 
        targets=targets.to(device)

        pred = model(inputs1,inputs2)

        # calcolo loss e metrica
        vloss = loss_func(pred, targets)
        vmetric = metric_func(pred, targets)

        vali_loss += loss.item()
        vali_metric += metric.item() 

    vali_loss /= counter
    vali_metric /= (counter*batch_size)
    hist_vloss.append(vali_loss)
    hist_vmetric.append(vali_metric)


    elapsed_time = time.time()-t0
    print("epoch: %d, time(s): %.4f, train loss: %.6f, train metric: %.6f, vali loss: %.6f, vali metric: %.6f" 
          % (epoch+1, elapsed_time, train_loss, train_metric, vali_loss, vali_metric))
    scheduler.step()

epoch: 1, time(s): 20.7488, train loss: 0.639498, train metric: 0.631720, vali loss: 0.632532, vali metric: 0.640000
epoch: 2, time(s): 21.8784, train loss: 0.568751, train metric: 0.706961, vali loss: 0.596349, vali metric: 0.690000
epoch: 3, time(s): 20.4080, train loss: 0.544166, train metric: 0.724384, vali loss: 0.565313, vali metric: 0.710000
epoch: 4, time(s): 19.6999, train loss: 0.531818, train metric: 0.732796, vali loss: 0.448046, vali metric: 0.820000
epoch: 5, time(s): 19.5034, train loss: 0.523045, train metric: 0.738265, vali loss: 0.502660, vali metric: 0.730000
epoch: 6, time(s): 19.5843, train loss: 0.516938, train metric: 0.742197, vali loss: 0.478394, vali metric: 0.800000
epoch: 7, time(s): 19.9673, train loss: 0.512275, train metric: 0.744937, vali loss: 0.555518, vali metric: 0.700000
epoch: 8, time(s): 20.6271, train loss: 0.508898, train metric: 0.747818, vali loss: 0.548215, vali metric: 0.770000
epoch: 9, time(s): 19.7401, train loss: 0.506469, train metric: 

In [ ]:
def ewma_vectorized(data, alpha, offset=None, dtype=None, order='C', out=None):
    """
    Calculates the exponential moving average over a vector.
    Will fail for large inputs.
    :param data: Input data
    :param alpha: scalar float in range (0,1)
        The alpha parameter for the moving average.
    :param offset: optional
        The offset for the moving average, scalar. Defaults to data[0].
    :param dtype: optional
        Data type used for calculations. Defaults to float64 unless
        data.dtype is float32, then it will use float32.
    :param order: {'C', 'F', 'A'}, optional
        Order to use when flattening the data. Defaults to 'C'.
    :param out: ndarray, or None, optional
        A location into which the result is stored. If provided, it must have
        the same shape as the input. If not provided or `None`,
        a freshly-allocated array is returned.
    """
    data = np.array(data, copy=False)

    if dtype is None:
        if data.dtype == np.float32:
            dtype = np.float32
        else:
            dtype = np.float64
    else:
        dtype = np.dtype(dtype)

    if data.ndim > 1:
        # flatten input
        data = data.reshape(-1, order)

    if out is None:
        out = np.empty_like(data, dtype=dtype)
    else:
        assert out.shape == data.shape
        assert out.dtype == dtype

    if data.size < 1:
        # empty input, return empty array
        return out

    if offset is None:
        offset = data[0]

    alpha = np.array(alpha, copy=False).astype(dtype, copy=False)

    # scaling_factors -> 0 as len(data) gets large
    # this leads to divide-by-zeros below
    scaling_factors = np.power(1. - alpha, np.arange(data.size + 1, dtype=dtype),
                               dtype=dtype)
    # create cumulative sum array
    np.multiply(data, (alpha * scaling_factors[-2]) / scaling_factors[:-1],
                dtype=dtype, out=out)
    np.cumsum(out, dtype=dtype, out=out)

    # cumsums / scaling
    out /= scaling_factors[-2::-1]

    if offset != 0:
        offset = np.array(offset, copy=False).astype(dtype, copy=False)
        # add offsets
        out += offset * scaling_factors[1:]

    return out

In [ ]:
# plot loss vs epoca
plt.figure(figsize=(10, 7))
ewma_hist_vloss = ewma_vectorized(hist_vloss,0.1)
plt.plot(range(1,len(hist_loss)+1), hist_loss, color='green', linestyle='-', label='Train Loss')
plt.plot(range(1,len(hist_vloss)+1), ewma_hist_vloss, color='blue', linestyle='-', label='Moving Average Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# plot accuracy vs epoca
plt.figure(figsize=(10, 7))
ewma_hist_vmetric = ewma_vectorized(hist_vmetric,0.1)
plt.plot(range(1,len(hist_metric)+1),hist_metric, color='green', linestyle='-', label='Train Metric')
plt.plot(range(1,len(hist_vmetric)+1),ewma_hist_vmetric, color='blue', linestyle='-', label='Moving Average Validation Metric')
plt.xlabel('Epochs')
plt.ylabel('Metric')
plt.legend()
plt.show()

In [ ]:
# Salvataggio del modello

torch.save(model, './trained_model.pt')

In [ ]:
# Uso di un modello addestrato in inferenza (predizione) 

# lettura del modello

model = torch.load('./trained_model.pt')
model.eval() #va sempre fatto per settare eventuali layer di droput e batch norm in evaluation model

# Non è strettamento necessario girare sulla GPU per l'inferenza (non richiede grande potenza di calcolo)

loss = 0
metric = 0
counter = 0

for inputs1, inputs2, targets in tes_dataloader:
  counter +=1
  inputs1=inputs1.to(device) 
  inputs2=inputs2.to(device) 
  targets=targets.to(device)
  pred = model(inputs1,inputs2)

  loss += loss_func(pred, targets).item()
  metric += accuracy(pred, targets).item()


loss /= counter
metric /= (counter*batch_size)

print('Test loss/accuracy: ',loss, ' / ', metric)


In [ ]:
#confusion matrix

model.to(torch.device('cpu'))

predictions = np.empty(shape=(0,))
truths = np.empty(shape=(0,))
 
for inputs1,inputs2,targets in tes_dataloader:
  counter += 1
  inputs1=inputs1.to('cpu')
  inputs2=inputs2.to('cpu')
  targets=targets.to('cpu')
  pred = model(inputs1,inputs2)

  res = torch.argmax(pred, dim=1)

  predictions = np.concatenate((predictions,res.detach().numpy()))
  truths = np.concatenate((truths,targets.detach().numpy()))

from sklearn.metrics import confusion_matrix
c_mat = confusion_matrix(predictions, truths, normalize='true')
print('Matrice di correlazione: ')
print(c_mat)


In [ ]:
from sklearn import metrics

fpr__0 = []
tpr__0 = []
roc_auc__0 = []

fpr__1 = []
tpr__1 = []
roc_auc__1 = []

for i in range(0,1000):

  inputs1,inputs2,targets = next(iter(tes_dataloader))

  pred=model(inputs1,inputs2)

  targets = targets.detach().numpy()
  pred = pred.detach().numpy()

  fpr_0, tpr_0, thresholds_0 = metrics.roc_curve(targets, pred[:,0], pos_label=0, drop_intermediate=False)
  roc_auc_0 = metrics.auc(fpr_0, tpr_0)

  fpr_1, tpr_1, thresholds_1 = metrics.roc_curve(targets, pred[:,1], pos_label=1, drop_intermediate=False)
  roc_auc_1 = metrics.auc(fpr_1, tpr_1)

  fpr__0.append(fpr_0)
  tpr__0.append(tpr_0)
  roc_auc__0.append(roc_auc_0)

  fpr__1.append(fpr_1)
  tpr__1.append(tpr_1)
  roc_auc__1.append(roc_auc_1)


print(f'label 0: ROC mean = {np.mean(roc_auc__0)}    Standard deviation = {np.std(roc_auc__0)}')
print(f'label 1: ROC mean = {np.mean(roc_auc__1)}    Standard deviation = {np.std(roc_auc__1)}')

In [ ]:
fpr_0_mean = [sum(x)/len(x) for x in zip(*fpr__0)]
fpr_1_mean = [sum(x)/len(x) for x in zip(*fpr__1)]
tpr_0_mean = [sum(x)/len(x) for x in zip(*tpr__0)]
tpr_1_mean = [sum(x)/len(x) for x in zip(*tpr__1)]

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr_0_mean, tpr_0_mean, color='sandybrown',
 lw=lw, label='Average ROC curve label 0 (Average AUC = %0.2f)' % np.mean(roc_auc__0))
plt.plot(fpr_1_mean, tpr_1_mean, color='limegreen',
 lw=lw, label='Average ROC curve label 1 (Average AUC = %0.2f)' % np.mean(roc_auc__1))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()